In [31]:
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
import numpy as np
import os
import json
import csv
import time
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F

In [39]:
WINDOW_SIZE = 1440
LEARNING_RATE = 1e-5
EPOCHS = 10
NUMBER_OF_WORKERS_PYTORCH = 4
q=3

In [47]:
def spectral_residual(values, n=q):
    """
    This method transform a time series into spectral residual series
    :param values: list.
        a list of float values.
    :return: mag: list.
        a list of float values as the spectral residual values
    """
    EPS = 1e-8
    trans = np.fft.fft(values)
    mag = np.sqrt(trans.real ** 2 + trans.imag ** 2)
    eps_index = np.where(mag <= EPS)[0]
    mag[eps_index] = EPS

    mag_log = np.log(mag)
    mag_log[eps_index] = 0

    spectral = np.exp(mag_log - average_filter(mag_log, n))

    trans.real = trans.real * spectral / mag
    trans.imag = trans.imag * spectral / mag
    trans.real[eps_index] = 0
    trans.imag[eps_index] = 0

    wave_r = np.fft.ifft(trans)
    mag = np.sqrt(wave_r.real ** 2 + wave_r.imag ** 2)
    return mag

def average_filter(values, n=3):
    """
    Calculate the sliding window average for the give time series.
    Mathematically, res[i] = sum_{j=i-t+1}^{i} values[j] / t, where t = min(n, i+1)
    :param values: list.
        a list of float numbers
    :param n: int, default 3.
        window size.
    :return res: list.
        a list of value after the average_filter process.
    """

    if n >= len(values):
        n = len(values)

    res = np.cumsum(values, dtype=float)
    res[n:] = res[n:] - res[:-n]
    res[n:] = res[n:] / n

    for i in range(1, n):
        res[i] /= (i + 1)

    return res

def predict_next(values):
    """
    Predicts the next value by sum up the slope of the last value with previous values.
    Mathematically, g = 1/m * sum_{i=1}^{m} g(x_n, x_{n-i}), x_{n+1} = x_{n-m+1} + g * m,
    where g(x_i,x_j) = (x_i - x_j) / (i - j)
    :param values: list.
        a list of float numbers.
    :return : float.
        the predicted next value.
    """

    if len(values) <= 1:
        raise ValueError(f'data should contain at least 2 numbers')

    v_last = values[-1]
    n = len(values)

    slopes = [(v_last - v) / (n - 1 - i) for i, v in enumerate(values[:-1])]

    return values[1] + sum(slopes)

def extend_series(values, extend_num=5, look_ahead=5):
    """
    extend the array data by the predicted next value
    :param values: list.
        a list of float numbers.
    :param extend_num: int, default 5.
        number of values added to the back of data.
    :param look_ahead: int, default 5.
        number of previous values used in prediction.
    :return: list.
        The result array.
    """

    if look_ahead < 1:
        raise ValueError('look_ahead must be at least 1')

    extension = [predict_next(values[-look_ahead - 2:-1])] * extend_num
    return np.concatenate((values, extension), axis=0)

class Anomaly(nn.Module):
    def __init__(self, window=1024):
        self.window = window
        super(Anomaly, self).__init__()
        self.layer1 = nn.Conv1d(window, window, kernel_size=1, stride=1, padding=0)
        self.layer2 = nn.Conv1d(window, 2 * window, kernel_size=1, stride=1, padding=0)
        self.fc1 = nn.Linear(2 * window, 4 * window)
        self.fc2 = nn.Linear(4 * window, window)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = x.view(x.size(0), self.window, 1)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return torch.sigmoid(x)
    
def cuda_if_available(x):
    if torch.cuda.is_available():
        return x.cuda()
    else:
        return x.cpu()

def calc(pred, true):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for pre, gt in zip(pred, true):
        if gt == 1:
            if pre == 1:
                TP += 1
            else:
                FN += 1
        if gt == 0:
            if pre == 1:
                FP += 1
            else:
                TN += 1
    print('TP=%d FP=%d TN=%d FN=%d' % (TP, FP, TN, FN))
    return TP, FP, TN, FN

In [41]:
kpis = {}
DATA_PATH=os.getcwd()+'/../sr-cnn/train.csv'
anomalies = 0
with open(DATA_PATH) as f:
    input = csv.reader(f, delimiter=',')
    cnt = 0
    for row in input:
        if cnt == 0:
            cnt += 1
            continue
        kpi = kpis.get(str(row[3]), [[],[],[]])
        kpi[0].append(int(row[0])) #timestamp
        kpi[1].append(float(row[1])) #value
        kpi[2].append(int(row[2])) #label
        kpis[str(row[3])] = kpi
        cnt += 1
        if int(row[2]) == 1:
            anomalies += 1
    f.close()

In [42]:
print('total number of anomalies in training data:', anomalies)

total number of anomalies in training data: 79554


to inject anomalous points according to the formula in the paper:

In [43]:
class gen():
    def __init__(self, win_siz, step, nums):
        self.control = 0
        self.win_siz = win_siz
        self.step = step
        self.number = nums

    def generate_train_data(self, value, back_k=0):
        def normalize(a):
            amin = np.min(a)
            amax = np.max(a)
            a = (a - amin) / (amax - amin + 1e-5)
            return 3 * a

        if back_k <= 5:
            back = back_k
        else:
            back = 5
        length = len(value)
        tmp = []
        for pt in range(self.win_siz, length - back, self.step):
            head = max(0, pt - self.win_siz)
            tail = min(length - back, pt)
            data = np.array(value[head:tail])
            data = data.astype(np.float64)
            data = normalize(data)
            num = np.random.randint(1, self.number)
            ids = np.random.choice(self.win_siz, num, replace=False)
            lbs = np.zeros(self.win_siz, dtype=np.int64)
            if (self.win_siz - 6) not in ids:
                self.control += np.random.random()
            else:
                self.control = 0
            if self.control > 100:
                ids[0] = self.win_siz - 6
                self.control = 0
            mean = np.mean(data)
            dataavg = average_filter(data)
            var = np.var(data)
            for id in ids:
                data[id] += (dataavg[id] + mean) * np.random.randn() * min((1 + var), 10)
                lbs[id] = 1
            tmp.append([data.tolist(), lbs.tolist()])
        return tmp

In [9]:
MAX_NUMBER_OF_ANOMALOUS_POINTS_TO_ADD = 10

data_with_anomalies_added = {}
generator = gen(WINDOW_SIZE, 128, MAX_NUMBER_OF_ANOMALOUS_POINTS_TO_ADD)
for kpi_id in kpis:
    train_kpi = kpis[kpi_id]
    in_timestamp = train_kpi[0]
    in_value = train_kpi[1]
    in_label = train_kpi[2]
    train_data = generator.generate_train_data(in_value)
    data_with_anomalies_added[kpi_id] = train_data

In [11]:
def adjust_lr(optimizer, epoch):
    cur_lr = LEARNING_RATE * (0.5 ** ((epoch + 10) // 10))
    for param in optimizer.param_groups:
        param['lr'] = cur_lr

def Var(x):
    return Variable(cuda_if_available(x))

def loss_function(x, lb, win_size=WINDOW_SIZE):
    l2_reg = 0.
    l2_weight = 0.
    for W in net.parameters():
        l2_reg = l2_reg + W.norm(2)
    kpiweight = torch.ones(lb.shape)
    kpiweight[lb == 1] = win_size // 100
    kpiweight = cuda_if_available(kpiweight)
    BCE = F.binary_cross_entropy(x, lb, weight=kpiweight, reduction='sum')
    return l2_reg * l2_weight + BCE

gen_set is to apply Spectral Residual to the training data and then provide it for later step (CNN).

In [13]:
class gen_set(Dataset):
    def __init__(self, width, data):
        self.genlen = 0
        self.len = self.genlen
        self.width = width
        self.kpinegraw = data
        self.negrawlen = len(self.kpinegraw)
        print('length :', len(self.kpinegraw))
        self.len += self.negrawlen
        self.kpineglen = 0
        self.control = 0.

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        idx = index % self.negrawlen
        datas = self.kpinegraw[idx]
        datas = np.array(datas)
        data = datas[0, :].astype(np.float64)
        lbs = datas[1, :].astype(np.float64)
        wave = spectral_residual(data)
        waveavg = average_filter(wave)
        for i in range(self.width):
            if wave[i] < 0.001 and waveavg[i] < 0.001:
                lbs[i] = 0
                continue
            ratio = wave[i] / waveavg[i]
            if ratio < 1.0 and lbs[i] == 1:
                lbs[i] = 0
            if ratio > 5.0:
                lbs[i] = 1
        srscore = abs(wave - waveavg) / (waveavg + 0.01)
        sortid = np.argsort(srscore)
        for idx in sortid[-2:]:
            if srscore[idx] > 3:
                lbs[idx] = 1
        resdata = torch.from_numpy(100 * wave)
        reslb = torch.from_numpy(lbs)
        return resdata, reslb

In [15]:
gen_data = {}
for kpi_id in data_with_anomalies_added:
    kpi = data_with_anomalies_added[kpi_id]
    gen_data[kpi_id] = gen_set(WINDOW_SIZE, kpi)

length : 831
length : 58
length : 76
length : 59
length : 58
length : 58
length : 58
length : 58
length : 997
length : 996
length : 997
length : 995
length : 995
length : 995
length : 997
length : 1132
length : 1132
length : 1132
length : 1132
length : 1123
length : 834
length : 834
length : 821
length : 1131
length : 1131
length : 1132
length : 1132
length : 1132
length : 1132


In [50]:
import math
def split_data(data):
    training_final = []
    validation_final = []
    for kpi_id in data:
        kpi = data[kpi_id]
        n_train = math.ceil(len(kpi) *0.5)
        n_validate = len(kpi) - n_train
        train_data, validation_data = torch.utils.data.random_split(kpi,[n_train,n_validate])
        training_final += train_data
        validation_final += validation_data
    return training_final, validation_data

In [19]:
def train(epoch, net, training_set, validation, optimizer):
    train_loader = torch.utils.data.DataLoader(dataset=gen_set, shuffle=True, 
                                                 batch_size=256, pin_memory=True)
    net.train()
    train_loss = 0
    totTP, totFP, totTN, totFN = 0, 0, 0, 0
    threshold = 0.5
    for batch_idx, (inputs, lb) in enumerate(tqdm(train_loader, desc="Iteration")):
        optimizer.zero_grad()
        inputs = inputs.float()
        lb = lb.float()
        valueseq = Var(inputs)
        lb = Var(lb)
        output = net(valueseq)
        
        loss1 = loss_function(output, lb)
        loss1.backward()
        train_loss += loss1.item()
        optimizer.step()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 5.0)
        if batch_idx % 100 == 0:
            aa = output.detach().cpu().numpy().reshape(-1)
            res = np.zeros(aa.shape, np.int64)
            res[aa > threshold] = 1
            bb = lb.detach().cpu().numpy().reshape(-1)
            TP, FP, TN, FN = calc(res, bb)
            totTP += TP
            totFP += FP
            totTN += TN
            totFN += FN
            print('TP=%d FP=%d TN=%d FN=%d' % (TP, FP, TN, FN))
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss1.item() / len(inputs)))
            accuracy = (totTP + totTN) / (totTP+totFP+totTN+totFN) * 100.0
            print('Accuracy:', accuracy)

In [44]:

model = Anomaly(WINDOW_SIZE)

if torch.cuda.is_available():
    net = model.cuda()
    gpu_num = torch.cuda.device_count()
    net = torch.nn.DataParallel(net, list(range(gpu_num)))
    print(net)
else:
    net = model.cpu()

base_lr = LEARNING_RATE
bp_parameters = filter(lambda p: p.requires_grad, net.parameters()) # back propagation parameters
optimizer = optim.SGD(bp_parameters, lr=base_lr, momentum=0.9, weight_decay=0.0)

#import math
#n_train = math.ceil(len(training_final) *0.5)
#n_validate = len(training_final) - n_train
#train_data, validation_data = torch.utils.data.random_split(training_final,[n_train,n_validate])
#gen_data = gen_set(WINDOW_SIZE, training_final)

for epoch in range(1, EPOCHS + 1):
    training, validation = split_data(gen_data)
    print('epoch :', epoch)
    train(epoch, net, training, validation, optimizer)
    adjust_lr(optimizer, epoch)

epoch : 1


TP=576 FP=177380 TN=190091 FN=593
TP=576 FP=177380 TN=190091 FN=593
Train Epoch: 1 [0/11585 (0%)]	Loss: 1038.012451
Accuracy: 51.72173394097223

epoch : 2


TP=20 FP=6380 TN=361040 FN=1200
TP=20 FP=6380 TN=361040 FN=1200
Train Epoch: 2 [0/11585 (0%)]	Loss: 9054.687500
Accuracy: 97.94379340277779

epoch : 3


TP=20 FP=6380 TN=361072 FN=1168
TP=20 FP=6380 TN=361072 FN=1168
Train Epoch: 3 [0/11585 (0%)]	Loss: 8879.687500
Accuracy: 97.95247395833333

epoch : 4


TP=23 FP=6377 TN=361011 FN=1229
TP=23 FP=6377 TN=361011 FN=1229
Train Epoch: 4 [0/11585 (0%)]	Loss: 9212.109375
Accuracy: 97.93674045138889

epoch : 5


...

TP=24 FP=6376 TN=361117 FN=1123
TP=24 FP=6376 TN=361117 FN=1123
Train Epoch: 6 [0/11585 (0%)]	Loss: 8632.031250
Accuracy: 97.96576605902779

epoch : 7


TP=23 FP=6377 TN=361040 FN=1200
TP=23 FP=6377 TN=361040 FN=1200
Train Epoch: 7 [0/11585 (0%)]	Loss: 9053.515625
Accuracy: 97.94460720486111

epoch : 8


TP=21 FP=6379 TN=361133 FN=1107
TP=21 FP=6379 TN=361133 FN=1107
Train Epoch: 8 [0/11585 (0%)]	Loss: 8545.703125
Accuracy: 97.96929253472221

epoch : 9


TP=19 FP=6381 TN=361098 FN=1142
TP=19 FP=6381 TN=361098 FN=1142
Train Epoch: 9 [0/11585 (0%)]	Loss: 8737.890625
Accuracy: 97.95925564236111

epoch : 10


TP=28 FP=6372 TN=361058 FN=1182
TP=28 FP=6372 TN=361058 FN=1182
Train Epoch: 10 [0/11585 (0%)]	Loss: 8953.125000
Accuracy: 97.95084635416667



# Testing

In [45]:
import pandas as pd
TEST_DATA_PATH=os.getcwd()+'/../sr-cnn/test.hdf'
test_data = pd.read_hdf(TEST_DATA_PATH)

In [46]:
z=21
extend_num= 5
tau = 3
backaddnum = 5
back = 0
step= 1
threshold = 0.95

def modelwork(x, net):
    with torch.no_grad():
        x = torch.from_numpy(100 * x).float()
        x = torch.unsqueeze(x, 0)
        x = Var(x)
        output = net(x)
    aa = output.detach().cpu().numpy().reshape(-1)
    res = np.zeros(aa.shape, np.int64)
    res[aa > threshold] = 1
    return res, aa

In [48]:
def predict(timestamp, value, label):
    length = len(timestamp)
    detres = [0] * (WINDOW_SIZE - backaddnum)
    scores = [0] * (WINDOW_SIZE - backaddnum)

    for pt in tqdm(range(WINDOW_SIZE - backaddnum + back + step, length - back, step)):
        head = max(0, pt - (WINDOW_SIZE - backaddnum))
        tail = min(length, pt)
        wave = np.array(extend_series(value[head:tail + back]))
        mag = spectral_residual(wave, z)
        modeloutput, rawout = modelwork(mag, net)
        for ipt in range(pt - step - back, pt - back):
            detres.append(modeloutput[ipt - head])
            scores.append(rawout[ipt - head].item())
    detres += [0] * (length - len(detres))
    scores += [0] * (length - len(scores))

    last = -1
    interval = min([timestamp[i] - timestamp[i - 1] for i in range(1, len(timestamp))])
    for i in range(1, len(timestamp)):
        if timestamp[i] - timestamp[i - 1] > interval:
            if last >= 0 and i - last < 1000:
                detres[i] = 1
                scores[i] = 1
        if detres[i] == 1:
            last = i

    return timestamp[:], label[:], detres[:], scores[:]

In [49]:
kpis = test_data.groupby(test_data["KPI ID"])
results = []
savedscore = []

for name, kpi in kpis:
    in_timestamp = kpi['timestamp'].tolist()
    in_value = kpi['value'].tolist()
    in_label = kpi['label'].tolist()
    timestamp, label, pre, scores = predict(in_timestamp, in_value, in_label)
    results.append([timestamp, label, pre, name])
    savedscore.append([label, scores, name, timestamp])

KeyboardInterrupt: 

In [26]:
def reconstruct_label(timestamp, label):
    timestamp = np.asarray(timestamp, np.int64)
    index = np.argsort(timestamp)

    timestamp_sorted = np.asarray(timestamp[index])
    interval = np.min(np.diff(timestamp_sorted))

    label = np.asarray(label, np.int64)
    label = np.asarray(label[index])

    idx = (timestamp_sorted - timestamp_sorted[0]) // interval

    new_label = np.zeros(shape=((timestamp_sorted[-1] - timestamp_sorted[0]) // interval + 1,), dtype=np.int64)
    new_label[idx] = label

    return new_label

In [27]:
def get_range_proba(predict, label, delay=7):
    predict = np.array(predict)
    label = np.array(label)

    splits = np.where(label[1:] != label[:-1])[0] + 1
    is_anomaly = label[0] == 1
    new_predict = np.array(predict)
    pos = 0

    for sp in splits:
        if is_anomaly:
            if 1 in predict[pos:min(pos + delay + 1, sp)]:
                new_predict[pos: sp] = 1
            else:
                new_predict[pos: sp] = 0
        is_anomaly = not is_anomaly
        pos = sp
    sp = len(label)

    if is_anomaly:
        if 1 in predict[pos: min(pos + delay + 1, sp)]:
            new_predict[pos: sp] = 1
        else:
            new_predict[pos: sp] = 0

    return new_predict

In [28]:
def reconstruct_series(timestamp, label, predict, delay=7):
    label = reconstruct_label(timestamp, label)
    predict = reconstruct_label(timestamp, predict)
    predict = get_range_proba(predict, label, delay)
    return label.tolist(), predict.tolist()

In [29]:
labels, predicts = [], []
delay=7
for timestamp, label, predict, _ in results:
    if timestamp == []:
        continue
    lbl, pdt = reconstruct_series(timestamp, label, predict, delay)
    labels += lbl
    predicts += pdt

In [30]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score(labels, predicts)
pre = precision_score(labels, predicts)
rec = recall_score(labels, predicts)
TP, FP, TN, FN = calc(predicts, labels)
print('precision', pre)
print('recall', rec)
print('f1', f1)

TP=3961 FP=1685 TN=3017311 FN=50599
precision 0.7015586255756288
recall 0.07259897360703813
f1 0.13158156994319503
